In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


In [2]:
df = pd.read_csv('retractions_error_of_analyses.csv')
df.tail(15)

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
1344,285,The solution structure of heparan sulfate diff...,(BLS) Biology - Molecular;(PHY) Chemistry;(PHY...,Department of Structural and Molecular Biology...,The Journal of Biological Chemistry,American Society for Biochemistry and Molecula...,United Kingdom,Sanaullah Khan;Elizabeth Rodriguez;Rima Patel;...,http://retractionwatch.com/2013/07/29/retracti...,Research Article;,7/19/2013 0:00,10.1074/jbc.A111.226027,23873987.0,5/16/2011 0:00,10.1074/jbc.M111.226027,21576246.0,Retraction,+Error in Analyses;+Error in Data;,No,NaN
1345,272,Over-expression of HSPA12B protects mice again...,(BLS) Biology - Cellular;(BLS) Biology - Molec...,Department of Thoracic and Cardiovascular Surg...,American Journal of Translational Research,e-Century Publishing Corporation,China,Yanjun Sun;Lincai Ye;Chuan Jiang;Jun Jiang;Hai...,NaN,Research Article;,3/15/2016 0:00,NaN,27186288.0,12/15/2015 0:00,NaN,26885270.0,Retraction,+Concerns/Issues About Data;+Error in Analyses...,No,NaN
1346,265,"Sedation with GPI 15715, a water-soluble prodr...",(HSC) Medicine - Anesthesia/Anaesthesia;(HSC) ...,"Department of Anesthesiology, University of Er...",Anesthesia & Analgesia,Wolters Kluwer - Lippincott Williams & Wilkins,Germany;United States,Jorg Fechner;Harald Ihmsen;Christine Schiessl;...,NaN,Clinical Study;,4/1/2010 0:00,10.1213/ANE.0b013e3181dbdb71,20357159.0,3/1/2005 0:00,10.1213/01.ANE.0000144772.13372.F4,15728055.0,Retraction,+Error by Third Party;+Error in Analyses;+Erro...,No,See also: https://journals.lww.com/anesthesia-...
1347,261,Cytosol has a small effect on protein backbone...,(BLS) Biochemistry;(BLS) Biology - Cellular;,"Department of Chemistry, Department of Biochem...",Biochemistry,American Chemical Society (ACS),United States,Julie E Bryant;Juliette T J Lecomte;Andrew L L...,NaN,Research Article;,6/13/2007 0:00,10.1021/bi700744h,17567051.0,8/1/2006 0:00,10.1021/bi060547b,1690676.0,Retraction,+Error in Analyses;+Error in Methods;+Results ...,No,NaN
1348,248,"N, S Co-doped Graphene Quantum Dots from a Sin...",(PHY) Chemistry;,Collaborative Innovation Center for Marine Bio...,Chemical Communications,Royal Society of Chemistry (RSC),China,Rijun Gui;Xifeng Liu;Hui Jin;Zonghua Wang;Feif...,http://retractionwatch.com/2015/05/27/scientis...,Research Article;,6/21/2015 0:00,10.1039/c5cc90244d,26008806.0,1/12/2015 0:00,10.1039/c4cc09280e,25578025.0,Retraction,+Concerns/Issues About Authorship/Affiliation;...,Yes,NaN
1349,247,Measuring Si-C60 Chemical Forces via Single Mo...,(PHY) Chemistry;,"School of Physics and Astronomy, University of...",Chemical Communications,Royal Society of Chemistry (RSC),United Kingdom,Cristina Chiutu;Andrew Stannard;Adam M Sweetma...,NaN,Research Article;,8/28/2014 0:00,10.1039/C4CC90295E,25055854.0,10/14/2011 0:00,10.1039/c1cc14147c,21869962.0,Retraction,+Error in Analyses;+Error in Results and/or Co...,Yes,NaN
1350,232,A Novel function of Poly(ADP-ribose) Polymeras...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,"Department of Community, Occupational & Family...",Cell Death and Differentiation,Nature Publishing Group,Singapore,Qing Huang;Y-T Wu;H-L Tan;C-N Ong;Han-Ming Shen,http://retractionwatch.com/2010/11/18/what-wer...,Research Article;,12/1/2010 0:00,10.1038/cdd.2010.121,20865014.0,2/16/2009 0:00,10.1038/cdd.2008.151,18974775.0,Retraction,+Error in Analyses;+Error in Data;,No,NaN
1351,231,Induced Endothelial Cells Enhance Osteogenesis...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,"Department of Orthopedics, Shanghai First Peop...","Cells, Tissues, Organs in Vivo, in Vitro (CTO)",Karger,China,Jie Tao;Yuan Sun;Qui-gen Wang,NaN,Research Article;,4/13/2010 0:00,10.1159/000313340,20408260.0,10/18/2008 0:00,10.1159/000218139,19420896.0,Retraction,+Error in Analyses;+Error in Data;+Re

In [3]:
print(df.shape)

(1359, 20)


In [4]:
df_paywalled_no = df[df['Paywalled'] == 'No']
df_paywalled_yes = df[df['Paywalled'] == 'Yes']

In [5]:
print(df_paywalled_yes.shape)
print(df_paywalled_no.shape)

(49, 20)
(1304, 20)


In [6]:
def get_pdf_from_doi(doi):
    doi_url = f"https://doi.org/{doi}"
    response = requests.get(doi_url)

    if response.status_code != 200:
        print(f"Failed to resolve DOI: {doi}")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    pdf_link = None

    for link in soup.find_all('a'):
        href = link.get('href', '')
        if 'pdf' in href.lower():
            pdf_link = href
            break
    
    if not pdf_link:
        print(f"Failed to find PDF link for DOI: {doi}")
        return None
    
    if not pdf_link.startswith('http'):
        pdf_link = response.url.rsplit('/', 1)[0] + '/' + pdf_link

    print(f"Downloading PDF from {pdf_link}")
    pdf_response = requests.get(pdf_link)

    if pdf_response.status_code == 200:
        pdf_filename = f"{doi.replace('/', '_')}.pdf"
        with open(pdf_filename, 'wb') as f:
            f.write(pdf_response.content)
        print(f"PDF downloaded successfully: {pdf_filename}")
    else:
        print(f"Failed to download PDF: {pdf_link}")



In [7]:
def download_pdfs_from_df(df, column_name):
    for index, row in df.iterrows():
        doi = row[column_name]
        if pd.notna(doi):  # Check if DOI is not NaN
            print(f"Processing DOI: {doi}")
            get_pdf_from_doi(doi)
        else:
            print(f"No DOI found for index {index}")

In [16]:
download_pdfs_from_df(df_paywalled_no, 'RetractionDOI')

Processing DOI: 10.1016/j.clnu.2022.04.021
Failed to find PDF link for DOI: 10.1016/j.clnu.2022.04.021
Processing DOI: 10.1177/00220426221121697
Failed to resolve DOI: 10.1177/00220426221121697
Processing DOI: 10.1177/00220426231222323
Failed to resolve DOI: 10.1177/00220426231222323
Processing DOI: 10.1093/jnci/djae217
Failed to resolve DOI: 10.1093/jnci/djae217
Processing DOI: 10.1186/s12889-024-19943-3
Failed to download PDF: https://bmcpublichealth.biomedcentral.com/articles/10.1186///bmcpublichealth.biomedcentral.com/counter/pdf/10.1186/s12889-024-19943-3.pdf
Processing DOI: 10.1371/journal.pone.0309771
Failed to find PDF link for DOI: 10.1371/journal.pone.0309771
Processing DOI: 10.1093/jeb/voae081
Failed to resolve DOI: 10.1093/jeb/voae081
Processing DOI: 10.1001/jamapediatrics.2024.2996
Failed to resolve DOI: 10.1001/jamapediatrics.2024.2996
Processing DOI: 10.1126/science.abb4382
Failed to resolve DOI: 10.1126/science.abb4382
Processing DOI: 10.1126/science.adr6434
Failed to r

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [8]:
import requests

def get_paper_by_doi(doi):
    # Define the base URL for Europe PMC's RESTful Web Service
    base_url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
    
    # Set up parameters for the query
    params = {
        "query": f"DOI:{doi}",  # Search by DOI
        "format": "json",        # Specify JSON format for the response
        "resultType": "core"     # Get full metadata for the publication
    }
    
    # Send a GET request to the API
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Check if there are results
        if data.get("resultList", {}).get("result"):
            return data["resultList"]["result"][0]  # Return the first result
        else:
            return f"No results found for DOI: {doi}"
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
doi = "10.1074/jbc.A111.226027"  # Replace with the DOI you are interested in
paper_info = get_paper_by_doi(doi)
print(paper_info)


{'id': '23873987', 'source': 'MED', 'pmid': '23873987', 'pmcid': 'PMC3774387', 'fullTextIdList': {'fullTextId': ['PMC3774387']}, 'doi': '10.1074/jbc.a111.226027', 'title': 'The solution structure of heparan sulfate differs from that of heparin: implications for function.', 'authorString': 'Khan S, Rodriguez E, Patel R, Gor J, Mulloy B, Perkins SJ.', 'authorList': {'author': [{'fullName': 'Khan S', 'firstName': 'Sanaullah', 'lastName': 'Khan', 'initials': 'S', 'authorId': {'type': 'ORCID', 'value': '0000-0003-2480-6239'}}, {'fullName': 'Rodriguez E', 'firstName': 'Elizabeth', 'lastName': 'Rodriguez', 'initials': 'E'}, {'fullName': 'Patel R', 'firstName': 'Rima', 'lastName': 'Patel', 'initials': 'R'}, {'fullName': 'Gor J', 'firstName': 'Jayesh', 'lastName': 'Gor', 'initials': 'J'}, {'fullName': 'Mulloy B', 'firstName': 'Barbara', 'lastName': 'Mulloy', 'initials': 'B'}, {'fullName': 'Perkins SJ', 'firstName': 'Stephen J', 'lastName': 'Perkins', 'initials': 'SJ'}]}, 'authorIdList': {'autho

In [11]:
import requests
import pandas as pd

# Assuming 'df' is your DataFrame with a 'RetractionDOI' column
def get_paper_by_doi(doi):
    base_url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
    params = {
        "query": f"DOI:{doi}",
        "format": "json",
        "resultType": "core"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("resultList", {}).get("result"):
            return data["resultList"]["result"][0]  # Return first result
        else:
            return None
    else:
        print(f"Error {response.status_code} for DOI {doi}")
        return None

# Fetch metadata for each DOI in the DataFrame and store results
def get_retraction_metadata(df_paywalled_no):
    metadata_results = []
    for doi in df_paywalled_no["RetractionDOI"].dropna():  # Skip NaN values
        metadata = get_paper_by_doi(doi)
        if metadata:
            metadata_results.append(metadata)
    
    # Convert results to DataFrame if needed
    metadata_df = pd.DataFrame(metadata_results)
    return metadata_df

# Usage
metadata_df = get_retraction_metadata(df)
print(metadata_df)


           id source      pmid                           doi  \
0    39343464    MED  39343464    10.1016/j.clnu.2022.04.021   
1    39267576    MED  39267576          10.1093/jnci/djae217   
2    39237928    MED  39237928    10.1186/s12889-024-19943-3   
3    39197035    MED  39197035  10.1371/journal.pone.0309771   
4    39145387    MED  39145387           10.1093/jeb/voae081   
..        ...    ...       ...                           ...   
914  17990378    MED  17990378    10.1016/j.cell.2007.01.001   
915  16220614    MED  16220614    10.1016/j.cell.2005.08.028   
916  22719271    MED  22719271     10.1107/s1600536812022787   
917  17567051    MED  17567051             10.1021/bi700744h   
918  26310234    MED  26310234        10.1021/acsami.5b07490   

                                                 title  \
0    Retraction notice to 'High protein diet improv...   
1    Retraction and Replacement of: Increasing powe...   
2    Retraction Note: Association of milk consumpti...   

In [12]:
print(metadata_df.columns)

Index(['id', 'source', 'pmid', 'doi', 'title', 'authorString', 'authorList',
       'journalInfo', 'pubYear', 'pageInfo', 'affiliation',
       'publicationStatus', 'language', 'pubModel', 'pubTypeList',
       'subsetList', 'fullTextUrlList', 'commentCorrectionList',
       'isOpenAccess', 'inEPMC', 'inPMC', 'hasPDF', 'hasBook', 'hasSuppl',
       'citedByCount', 'hasData', 'hasReferences', 'hasTextMinedTerms',
       'hasDbCrossReferences', 'hasLabsLinks', 'hasEvaluations', 'authMan',
       'epmcAuthMan', 'nihAuthMan', 'hasTMAccessionNumbers', 'dateOfCreation',
       'firstIndexDate', 'dateOfRevision', 'firstPublicationDate', 'pmcid',
       'fullTextIdList', 'dataLinksTagsList', 'license',
       'fullTextReceivedDate', 'electronicPublicationDate', 'authorIdList',
       'abstractText', 'dateOfCompletion', 'meshHeadingList', 'chemicalList',
       'grantsList', 'manuscriptId', 'bookOrReportDetails',
       'tmAccessionTypeList', 'keywordList', 'dbCrossReferenceList',
       'embar

In [13]:
nan_titles = df[df['Title'].isna()]
print(nan_titles)

Empty DataFrame
Columns: [Record ID, Title, Subject, Institution, Journal, Publisher, Country, Author, URLS, ArticleType, RetractionDate, RetractionDOI, RetractionPubMedID, OriginalPaperDate, OriginalPaperDOI, OriginalPaperPubMedID, RetractionNature, Reason, Paywalled, Notes]
Index: []


In [14]:
pd.set_option('display.max_columns', None)
metadata_df.head()

,id,source,pmid,doi,title,authorString,authorList,journalInfo,pubYear,pageInfo,affiliation,publicationStatus,language,pubModel,pubTypeList,subsetList,fullTextUrlList,commentCorrectionList,isOpenAccess,inEPMC,inPMC,hasPDF,hasBook,hasSuppl,citedByCount,hasData,hasReferences,hasTextMinedTerms,hasDbCrossReferences,hasLabsLinks,hasEvaluations,authMan,epmcAuthMan,nihAuthMan,hasTMAccessionNumbers,dateOfCreation,firstIndexDate,dateOfRevision,firstPublicationDate,pmcid,fullTextIdList,dataLinksTagsList,license,fullTextReceivedDate,electronicPublicationDate,authorIdList,abstractText,dateOfCompletion,meshHeadingList,chemicalList,grantsList,manuscriptId,bookOrReportDetails,tmAccessionTypeList,keywordList,dbCrossReferenceList,embargoDate,investigatorList
0,39343464,MED,39343464,10.1016/j.clnu.2022.04.021,Retraction notice to 'High protein diet improv...,"Pimentel GD, Pichard C, Laviano A, Fernandes RC.","{'author': [{'fullName': 'Pimentel GD', 'first...","{'issue': '10', 'volume': '43', 'journalIssueI...",2024,2473,Clinical and Sports Nutrition Research Laborat...,ppublish,eng,Print,"{'pubType': ['Retraction of Publication', 'Jou...","{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '32919817', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2024-09-29,2024-09-30,2024-09-29,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39267576,MED,39267576,10.1093/jnci/djae217,Retraction and Replacement of: Increasing powe...,NaN,NaN,"{'issue': '10', 'volume': '116', 'journalIssue...",2024,1706,NaN,ppublish,eng,Print,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '38603624', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2024-09-13,2024-09-13,2024-10-09,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39237928,MED,39237928,10.1186/s12889-024-19943-3,Retraction Note: Association of milk consumpti...,"Ma Y, Liu Y, Jia F.","{'author': [{'fullName': 'Ma Y', 'firstName': ...","{'issue': '1', 'volume': '24', 'journalIssueId...",2024,2423,Department of Hepatobiliary and Pancreatic Sur...,epublish,eng,Electronic,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '37635228', 'sou...",Y,Y,Y,Y,N,N,0,N,N,Y,N,Y,N,N,N,N,N,2024-09-05,2024-09-06,2024-09-08,2024-09-05,PMC11378499,{'fullTextId': ['PMC11378499']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-09,2024-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39197035,MED,39197035,10.1371/journal.pone.0309771,Retraction: Analysis of the factors influencin...,PLOS ONE Editors.,{'author': [{'collectiveName': 'PLOS ONE Edito...,"{'issue': '8', 'volume': '19', 'journalIssueId...",2024,e0309771,NaN,epublish,eng,Electronic-eCollection,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '36520835', 'sou...",Y,Y,Y,Y,N,N,0,N,Y,N,N,Y,N,N,N,N,N,2024-08-28,2024-08-30,2024-08-31,2024-08-28,PMC11356393,{'fullTextId': ['PMC11356393']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-03,2024-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,39145387,MED,39145387,10.1093/jeb/voae081,Retraction and replacement of: Mitochondrial b...,NaN,NaN,"{'issue': '9', 'volume': '37', 'journalIssueId...",2024,1134,NaN,ppublish,eng,Print,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '38456649', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2024-08-15,2024-08-15,2024-08-27,2024-08-01,NaN,NaN,NaN,cc by,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#print full text url list
print(metadata_df['fullTextUrlList'])

0      {'fullTextUrl': [{'availability': 'Subscriptio...
1      {'fullTextUrl': [{'availability': 'Subscriptio...
2      {'fullTextUrl': [{'availability': 'Open access...
3      {'fullTextUrl': [{'availability': 'Open access...
4      {'fullTextUrl': [{'availability': 'Open access...
                             ...                        
914    {'fullTextUrl': [{'availability': 'Free', 'ava...
915    {'fullTextUrl': [{'availability': 'Free', 'ava...
916    {'fullTextUrl': [{'availability': 'Open access...
917    {'fullTextUrl': [{'availability': 'Subscriptio...
918    {'fullTextUrl': [{'availability': 'Subscriptio...
Name: fullTextUrlList, Length: 919, dtype: object


In [16]:
#show first 5 rows but entire text
print(metadata_df['fullTextUrlList'].head())

0    {'fullTextUrl': [{'availability': 'Subscriptio...
1    {'fullTextUrl': [{'availability': 'Subscriptio...
2    {'fullTextUrl': [{'availability': 'Open access...
3    {'fullTextUrl': [{'availability': 'Open access...
4    {'fullTextUrl': [{'availability': 'Open access...
Name: fullTextUrlList, dtype: object


In [17]:
#saveb this to a csv
metadata_df.to_csv('metadata_retractions.csv', index=False)

In [5]:
#lets read metadata_retractions.csv
metadata_retractions = pd.read_csv('metadata_retractions.csv')

In [18]:
metadata_df['Retraction_Url'] = metadata_df['fullTextUrlList'].apply(lambda x: x['fullTextUrl'][0]['url'] if 'fullTextUrl' in x and x['fullTextUrl'] else None)

In [16]:
pd.set_option('display.max_columns', None)
metadata_df.head()

,id,source,pmid,doi,title,authorString,authorList,journalInfo,pubYear,pageInfo,affiliation,publicationStatus,language,pubModel,pubTypeList,subsetList,fullTextUrlList,commentCorrectionList,isOpenAccess,inEPMC,inPMC,hasPDF,hasBook,hasSuppl,citedByCount,hasData,hasReferences,hasTextMinedTerms,hasDbCrossReferences,hasLabsLinks,hasEvaluations,authMan,epmcAuthMan,nihAuthMan,hasTMAccessionNumbers,dateOfCreation,firstIndexDate,dateOfRevision,firstPublicationDate,pmcid,fullTextIdList,dataLinksTagsList,license,fullTextReceivedDate,electronicPublicationDate,authorIdList,abstractText,dateOfCompletion,meshHeadingList,chemicalList,grantsList,manuscriptId,bookOrReportDetails,tmAccessionTypeList,keywordList,dbCrossReferenceList,embargoDate,investigatorList,Retraction_Url
0,39343464,MED,39343464,10.1016/j.clnu.2022.04.021,Retraction notice to 'High protein diet improv...,"Pimentel GD, Pichard C, Laviano A, Fernandes RC.","{'author': [{'fullName': 'Pimentel GD', 'first...","{'issue': '10', 'volume': '43', 'journalIssueI...",2024,2473,Clinical and Sports Nutrition Research Laborat...,ppublish,eng,Print,"{'pubType': ['Retraction of Publication', 'Jou...","{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '32919817', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2024-09-29,2024-09-30,2024-09-29,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1016/j.clnu.2022.04.021
1,39267576,MED,39267576,10.1093/jnci/djae217,Retraction and Replacement of: Increasing powe...,NaN,NaN,"{'issue': '10', 'volume': '116', 'journalIssue...",2024,1706,NaN,ppublish,eng,Print,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '38603624', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2024-09-13,2024-09-13,2024-10-09,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1093/jnci/djae217
2,39237928,MED,39237928,10.1186/s12889-024-19943-3,Retraction Note: Association of milk consumpti...,"Ma Y, Liu Y, Jia F.","{'author': [{'fullName': 'Ma Y', 'firstName': ...","{'issue': '1', 'volume': '24', 'journalIssueId...",2024,2423,Department of Hepatobiliary and Pancreatic Sur...,epublish,eng,Electronic,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '37635228', 'sou...",Y,Y,Y,Y,N,N,0,N,N,Y,N,Y,N,N,N,N,N,2024-09-05,2024-09-06,2024-09-08,2024-09-05,PMC11378499,{'fullTextId': ['PMC11378499']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-09,2024-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1186/s12889-024-19943-3
3,39197035,MED,39197035,10.1371/journal.pone.0309771,Retraction: Analysis of the factors influencin...,PLOS ONE Editors.,{'author': [{'collectiveName': 'PLOS ONE Edito...,"{'issue': '8', 'volume': '19', 'journalIssueId...",2024,e0309771,NaN,epublish,eng,Electronic-eCollection,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '36520835', 'sou...",Y,Y,Y,Y,N,N,0,N,Y,N,N,Y,N,N,N,N,N,2024-08-28,2024-08-30,2024-08-31,2024-08-28,PMC11356393,{'fullTextId': ['PMC11356393']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-03,2024-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1371/journal.pone.0309771
4,39145387,MED,39145387,10.1093/jeb/voae081,Retraction and replacement of: Mitochondrial b...,NaN,NaN,"{'issue': '9', 'volume': '37', 'journalIssueId...",2024,1134,NaN,ppublish,eng,Print,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '38456649', 'sou.

In [21]:
# I want tp look at all elements in Retraction_Url, go to the page and download the pdf
import requests
from bs4 import BeautifulSoup
import time

def download_pdf_from_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        
        # Check if URL ends with .pdf
        if url.endswith('.pdf'):
            pdf_filename = url.split('/')[-1]
            with open(pdf_filename, 'wb') as f:
                f.write(response.content)
            print(f"PDF downloaded successfully: {pdf_filename}")
        
        # If it's a DOI link, navigate to the page to find the actual PDF link
        else:
            soup = BeautifulSoup(response.content, 'html.parser')
            pdf_link = None
            
            # Try to find a PDF download link
            for link in soup.find_all('a', href=True):
                if '.pdf' in link['href']:
                    pdf_link = link['href']
                    break
            
            if pdf_link:
                # If the PDF link is relative, construct the full URL
                if not pdf_link.startswith('http'):
                    pdf_link = requests.compat.urljoin(url, pdf_link)
                
                # Download the PDF from the found link
                pdf_response = requests.get(pdf_link, headers=headers, timeout=10)
                pdf_filename = pdf_link.split('/')[-1]
                with open(pdf_filename, 'wb') as f:
                    f.write(pdf_response.content)
                print(f"PDF downloaded successfully: {pdf_filename}")
            else:
                print(f"No PDF link found on DOI page: {url}")
        
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {url}: {e}")
    time.sleep(1)  # To avoid overwhelming servers

# Download PDFs from the URLs
for url in metadata_df['Retraction_Url'].dropna():
    download_pdf_from_url(url)


No PDF link found on DOI page: https://doi.org/10.1016/j.clnu.2022.04.021
PDF downloaded successfully: djae217.pdf
PDF downloaded successfully: s12889-024-19943-3.pdf
No PDF link found on DOI page: https://doi.org/10.1371/journal.pone.0309771


KeyboardInterrupt: 

In [21]:
import requests
from bs4 import BeautifulSoup
import os

def download_pdf_from_url(url, save_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }

    try:
        # Start the session and get initial response
        session = requests.Session()
        response = session.get(url, headers=headers, allow_redirects=True)
        
        # Check if the response content-type is a PDF
        if response.headers['Content-Type'] == 'application/pdf':
            # Direct PDF download
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"PDF downloaded directly to {save_path}")
            return

        # Otherwise, parse the HTML to find a PDF download link
        soup = BeautifulSoup(response.text, 'html.parser')

        # Attempt to find a download button
        download_link = None
        for a_tag in soup.find_all('a', href=True):
            if 'pdf' in a_tag.get('href').lower():
                download_link = a_tag['href']
                break

        if not download_link:
            print("PDF download link not found.")
            return

        # Complete the URL if necessary
        if not download_link.startswith("http"):
            base_url = "/".join(url.split("/")[:3])
            download_link = base_url + download_link
            print(f"Constructed download link: {download_link}")

        # Download the PDF from the found link
        pdf_response = session.get(download_link, headers=headers)
        if pdf_response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(pdf_response.content)
            print(f"PDF downloaded from link to {save_path}")
        else:
            print("Failed to download PDF from the download link.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [22]:
for i, url in metadata_df['Retraction_Url'].dropna().items():
    save_path = f"retraction_{i}.pdf"  # Custom save path for each file
    download_pdf_from_url(url, save_path)

PDF download link not found.
Constructed download link: https://doi.org/jnci/article-pdf/116/10/1706/59634424/djae217.pdf
Failed to download PDF from the download link.
Constructed download link: https://doi.org//bmcpublichealth.biomedcentral.com/counter/pdf/10.1186/s12889-024-19943-3.pdf
Failed to download PDF from the download link.
PDF download link not found.
PDF downloaded directly to retraction_4.pdf
Constructed download link: https://doi.org/journals/jamapediatrics/articlepdf/2822489/jamapediatrics_mandhane_2024_rx_240001_1728048779.37294.pdf
Failed to download PDF from the download link.
PDF download link not found.
PDF download link not found.
PDF download link not found.
PDF downloaded directly to retraction_9.pdf
PDF download link not found.
PDF downloaded directly to retraction_11.pdf
PDF downloaded from link to retraction_12.pdf
Constructed download link: https://doi.org/journals/nutrition/articles/10.3389/fnut.2024.1465530/pdf
Failed to download PDF from the download link

/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


PDF download link not found.
PDF download link not found.
PDF downloaded from link to retraction_472.pdf
PDF download link not found.
PDF downloaded directly to retraction_474.pdf
PDF downloaded directly to retraction_475.pdf
PDF download link not found.
PDF downloaded directly to retraction_477.pdf
PDF download link not found.
PDF download link not found.
PDF download link not found.
PDF download link not found.
PDF downloaded directly to retraction_482.pdf
PDF download link not found.
PDF download link not found.
PDF downloaded directly to retraction_485.pdf
PDF downloaded directly to retraction_486.pdf
PDF downloaded directly to retraction_487.pdf
PDF downloaded directly to retraction_488.pdf
PDF downloaded directly to retraction_489.pdf
PDF downloaded from link to retraction_490.pdf
PDF download link not found.
PDF downloaded from link to retraction_492.pdf
PDF downloaded directly to retraction_493.pdf
PDF downloaded directly to retraction_494.pdf
PDF downloaded directly to retract

In [23]:
metadata_df.shape

(919, 59)

In [24]:
#get 4th row from metadata_df
metadata_df.iloc[4]

id                                                                    39145387
source                                                                     MED
pmid                                                                  39145387
doi                                                        10.1093/jeb/voae081
title                        Retraction and replacement of: Mitochondrial b...
authorString                                                               NaN
authorList                                                                 NaN
journalInfo                  {'issue': '9', 'volume': '37', 'journalIssueId...
pubYear                                                                   2024
pageInfo                                                                  1134
affiliation                                                                NaN
publicationStatus                                                     ppublish
language                                            

In [25]:
import pandas as pd
import os

# Assuming `metadata_df` is already loaded

# Step 1: Get all files in the current directory
existing_files = os.listdir('.')

# Step 2: Filter files that start with "retraction_" and extract the numbers
retraction_numbers = [
    int(file.split('_')[1].split('.')[0])  # Extract the number after "retraction_"
    for file in existing_files
    if file.startswith("retraction_") and file.split('_')[1].split('.')[0].isdigit()
]

# Step 3: Create a new DataFrame excluding rows where the index is in `retraction_numbers`
metadata_download_remaining = metadata_df.drop(retraction_numbers)

# Display the new DataFrame to verify
metadata_download_remaining.head()


,id,source,pmid,doi,title,authorString,authorList,journalInfo,pubYear,pageInfo,affiliation,publicationStatus,language,pubModel,pubTypeList,subsetList,fullTextUrlList,commentCorrectionList,isOpenAccess,inEPMC,inPMC,hasPDF,hasBook,hasSuppl,citedByCount,hasData,hasReferences,hasTextMinedTerms,hasDbCrossReferences,hasLabsLinks,hasEvaluations,authMan,epmcAuthMan,nihAuthMan,hasTMAccessionNumbers,dateOfCreation,firstIndexDate,dateOfRevision,firstPublicationDate,pmcid,fullTextIdList,dataLinksTagsList,license,fullTextReceivedDate,electronicPublicationDate,authorIdList,abstractText,dateOfCompletion,meshHeadingList,chemicalList,grantsList,manuscriptId,bookOrReportDetails,tmAccessionTypeList,keywordList,dbCrossReferenceList,embargoDate,investigatorList,Retraction_Url
0,39343464,MED,39343464,10.1016/j.clnu.2022.04.021,Retraction notice to 'High protein diet improv...,"Pimentel GD, Pichard C, Laviano A, Fernandes RC.","{'author': [{'fullName': 'Pimentel GD', 'first...","{'issue': '10', 'volume': '43', 'journalIssueI...",2024,2473,Clinical and Sports Nutrition Research Laborat...,ppublish,eng,Print,"{'pubType': ['Retraction of Publication', 'Jou...","{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '32919817', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2024-09-29,2024-09-30,2024-09-29,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1016/j.clnu.2022.04.021
1,39267576,MED,39267576,10.1093/jnci/djae217,Retraction and Replacement of: Increasing powe...,NaN,NaN,"{'issue': '10', 'volume': '116', 'journalIssue...",2024,1706,NaN,ppublish,eng,Print,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '38603624', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2024-09-13,2024-09-13,2024-10-09,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1093/jnci/djae217
2,39237928,MED,39237928,10.1186/s12889-024-19943-3,Retraction Note: Association of milk consumpti...,"Ma Y, Liu Y, Jia F.","{'author': [{'fullName': 'Ma Y', 'firstName': ...","{'issue': '1', 'volume': '24', 'journalIssueId...",2024,2423,Department of Hepatobiliary and Pancreatic Sur...,epublish,eng,Electronic,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '37635228', 'sou...",Y,Y,Y,Y,N,N,0,N,N,Y,N,Y,N,N,N,N,N,2024-09-05,2024-09-06,2024-09-08,2024-09-05,PMC11378499,{'fullTextId': ['PMC11378499']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-09,2024-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1186/s12889-024-19943-3
3,39197035,MED,39197035,10.1371/journal.pone.0309771,Retraction: Analysis of the factors influencin...,PLOS ONE Editors.,{'author': [{'collectiveName': 'PLOS ONE Edito...,"{'issue': '8', 'volume': '19', 'journalIssueId...",2024,e0309771,NaN,epublish,eng,Electronic-eCollection,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '36520835', 'sou...",Y,Y,Y,Y,N,N,0,N,Y,N,N,Y,N,N,N,N,N,2024-08-28,2024-08-30,2024-08-31,2024-08-28,PMC11356393,{'fullTextId': ['PMC11356393']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-03,2024-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1371/journal.pone.0309771
5,39158902,MED,39158902,10.1001/jamapediatrics.2024.2996,"Notice of Retraction: Hahn LM, et al. Post-COV...",Mandhane PJ.,"{'author': [{'fullName': 'Mandhane PJ', 'first...","{'issue': '10', 'volume': '178', 'journalIssue...",2024,1085-1086,"Department of Pediatrics, Faculty of Medicine ...",ppublish,eng,Print,"{'pubType': ['Retraction of Publication', 'Jou...","{'subset': [{'c

In [26]:
metadata_download_remaining.shape

(566, 59)

In [27]:
metadata_download_remaining.to_csv('metadata_retractions.csv', index=False)

In [29]:
import requests
from bs4 import BeautifulSoup
import os

def follow_doi_and_download_pdf(doi_url, save_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }
    
    try:
        # Step 1: Follow DOI link
        session = requests.Session()
        response = session.get(doi_url, headers=headers, allow_redirects=True)
        
        # Check if it's a direct PDF link
        if response.headers['Content-Type'] == 'application/pdf':
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"PDF downloaded directly to {save_path}")
            return

        # Step 2: Parse the redirected page to find a PDF download link
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Look for links that contain 'pdf' in the href
        download_link = None
        for a_tag in soup.find_all('a', href=True):
            if 'pdf' in a_tag['href'].lower():
                download_link = a_tag['href']
                break
        
        # If a download link is found, check if it's a relative URL
        if download_link:
            if not download_link.startswith("http"):
                base_url = "/".join(response.url.split("/")[:3])  # Base URL of the redirected page
                download_link = base_url + download_link
            
            # Step 3: Download the PDF
            pdf_response = session.get(download_link, headers=headers)
            if pdf_response.status_code == 200:
                with open(save_path, 'wb') as f:
                    f.write(pdf_response.content)
                print(f"PDF downloaded from link to {save_path}")
            else:
                print("Failed to download PDF from the found link.")
        else:
            print("PDF download link not found on the page.")

    except Exception as e:
        print(f"An error occurred: {e}")

os.makedirs("BSIteration1", exist_ok=True)

# Loop over URLs in DataFrame and download each PDF
for i, url in metadata_download_remaining['Retraction_Url'].dropna().items():
    save_path = os.path.join("BSIteration1", f"retraction_{i}.pdf")  # Save path in the new folder
    follow_doi_and_download_pdf(url, save_path)


PDF download link not found on the page.
PDF downloaded from link to BSIteration1/retraction_1.pdf
Failed to download PDF from the found link.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF downloaded from link to BSIteration1/retraction_13.pdf
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF downloaded directly to BSIteration1/retraction_18.pdf
PDF downloaded from link to BSIteration1/retraction_19.pdf
PDF download link not found on the page.
PDF download link not found on the page.
PDF downloaded from link to BSIteration1/retraction_23.pdf
PDF downloaded from link to BSIteration1/retraction_24.pdf
PDF download link not found on the page.
PDF download link not found on

/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
Failed to download PDF from the found link.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF downloaded directly to BSIteration1/retraction_510.pdf
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the p

In [30]:
import pandas as pd
import os

# Assuming `metadata_download_remaining` is already loaded from the previous steps

# Step 1: Get all files in the "BSIteration1" folder
bs_iteration1_files = os.listdir('BSIteration1')

# Step 2: Filter files that start with "retraction_" and extract the numbers
retraction_numbers = [
    int(file.split('_')[1].split('.')[0])  # Extract the number after "retraction_"
    for file in bs_iteration1_files
    if file.startswith("retraction_") and file.split('_')[1].split('.')[0].isdigit()
]

# Step 3: Create a new DataFrame excluding rows where the index is in `retraction_numbers`
metadata_download_remaining_iteration2 = metadata_download_remaining.drop(retraction_numbers)

# Display the new DataFrame to verify
metadata_download_remaining_iteration2.head()


,id,source,pmid,doi,title,authorString,authorList,journalInfo,pubYear,pageInfo,affiliation,publicationStatus,language,pubModel,pubTypeList,subsetList,fullTextUrlList,commentCorrectionList,isOpenAccess,inEPMC,inPMC,hasPDF,hasBook,hasSuppl,citedByCount,hasData,hasReferences,hasTextMinedTerms,hasDbCrossReferences,hasLabsLinks,hasEvaluations,authMan,epmcAuthMan,nihAuthMan,hasTMAccessionNumbers,dateOfCreation,firstIndexDate,dateOfRevision,firstPublicationDate,pmcid,fullTextIdList,dataLinksTagsList,license,fullTextReceivedDate,electronicPublicationDate,authorIdList,abstractText,dateOfCompletion,meshHeadingList,chemicalList,grantsList,manuscriptId,bookOrReportDetails,tmAccessionTypeList,keywordList,dbCrossReferenceList,embargoDate,investigatorList,Retraction_Url
0,39343464,MED,39343464,10.1016/j.clnu.2022.04.021,Retraction notice to 'High protein diet improv...,"Pimentel GD, Pichard C, Laviano A, Fernandes RC.","{'author': [{'fullName': 'Pimentel GD', 'first...","{'issue': '10', 'volume': '43', 'journalIssueI...",2024,2473,Clinical and Sports Nutrition Research Laborat...,ppublish,eng,Print,"{'pubType': ['Retraction of Publication', 'Jou...","{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '32919817', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2024-09-29,2024-09-30,2024-09-29,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1016/j.clnu.2022.04.021
2,39237928,MED,39237928,10.1186/s12889-024-19943-3,Retraction Note: Association of milk consumpti...,"Ma Y, Liu Y, Jia F.","{'author': [{'fullName': 'Ma Y', 'firstName': ...","{'issue': '1', 'volume': '24', 'journalIssueId...",2024,2423,Department of Hepatobiliary and Pancreatic Sur...,epublish,eng,Electronic,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '37635228', 'sou...",Y,Y,Y,Y,N,N,0,N,N,Y,N,Y,N,N,N,N,N,2024-09-05,2024-09-06,2024-09-08,2024-09-05,PMC11378499,{'fullTextId': ['PMC11378499']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-09,2024-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1186/s12889-024-19943-3
3,39197035,MED,39197035,10.1371/journal.pone.0309771,Retraction: Analysis of the factors influencin...,PLOS ONE Editors.,{'author': [{'collectiveName': 'PLOS ONE Edito...,"{'issue': '8', 'volume': '19', 'journalIssueId...",2024,e0309771,NaN,epublish,eng,Electronic-eCollection,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '36520835', 'sou...",Y,Y,Y,Y,N,N,0,N,Y,N,N,Y,N,N,N,N,N,2024-08-28,2024-08-30,2024-08-31,2024-08-28,PMC11356393,{'fullTextId': ['PMC11356393']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-03,2024-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1371/journal.pone.0309771
5,39158902,MED,39158902,10.1001/jamapediatrics.2024.2996,"Notice of Retraction: Hahn LM, et al. Post-COV...",Mandhane PJ.,"{'author': [{'fullName': 'Mandhane PJ', 'first...","{'issue': '10', 'volume': '178', 'journalIssue...",2024,1085-1086,"Department of Pediatrics, Faculty of Medicine ...",ppublish,eng,Print,"{'pubType': ['Retraction of Publication', 'Jou...","{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '37721744', 'sou...",N,N,N,N,N,N,1,N,N,N,N,Y,N,N,N,N,N,2024-08-19,2024-08-22,2024-10-07,2024-10-01,NaN,NaN,{'dataLinkstag': ['altmetrics']},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1001/jamapediatrics.2024.2996
6,32139518,MED,32139518,10.1126/science.abb4382,"Erratum for the Report: ""A room-temperature si...",NaN,NaN,"{'issue': '6482', 'volume': '367', 'journalIss...",2020,eabb4382,NaN,ppublish,eng,Print,"{'pubTy

In [33]:
metadata_download_remaining_iteration2.head(10)

,id,source,pmid,doi,title,authorString,authorList,journalInfo,pubYear,pageInfo,affiliation,publicationStatus,language,pubModel,pubTypeList,subsetList,fullTextUrlList,commentCorrectionList,isOpenAccess,inEPMC,inPMC,hasPDF,hasBook,hasSuppl,citedByCount,hasData,hasReferences,hasTextMinedTerms,hasDbCrossReferences,hasLabsLinks,hasEvaluations,authMan,epmcAuthMan,nihAuthMan,hasTMAccessionNumbers,dateOfCreation,firstIndexDate,dateOfRevision,firstPublicationDate,pmcid,fullTextIdList,dataLinksTagsList,license,fullTextReceivedDate,electronicPublicationDate,authorIdList,abstractText,dateOfCompletion,meshHeadingList,chemicalList,grantsList,manuscriptId,bookOrReportDetails,tmAccessionTypeList,keywordList,dbCrossReferenceList,embargoDate,investigatorList,Retraction_Url
0,39343464,MED,39343464,10.1016/j.clnu.2022.04.021,Retraction notice to 'High protein diet improv...,"Pimentel GD, Pichard C, Laviano A, Fernandes RC.","{'author': [{'fullName': 'Pimentel GD', 'first...","{'issue': '10', 'volume': '43', 'journalIssueI...",2024,2473,Clinical and Sports Nutrition Research Laborat...,ppublish,eng,Print,"{'pubType': ['Retraction of Publication', 'Jou...","{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '32919817', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2024-09-29,2024-09-30,2024-09-29,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1016/j.clnu.2022.04.021
2,39237928,MED,39237928,10.1186/s12889-024-19943-3,Retraction Note: Association of milk consumpti...,"Ma Y, Liu Y, Jia F.","{'author': [{'fullName': 'Ma Y', 'firstName': ...","{'issue': '1', 'volume': '24', 'journalIssueId...",2024,2423,Department of Hepatobiliary and Pancreatic Sur...,epublish,eng,Electronic,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '37635228', 'sou...",Y,Y,Y,Y,N,N,0,N,N,Y,N,Y,N,N,N,N,N,2024-09-05,2024-09-06,2024-09-08,2024-09-05,PMC11378499,{'fullTextId': ['PMC11378499']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-09,2024-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1186/s12889-024-19943-3
3,39197035,MED,39197035,10.1371/journal.pone.0309771,Retraction: Analysis of the factors influencin...,PLOS ONE Editors.,{'author': [{'collectiveName': 'PLOS ONE Edito...,"{'issue': '8', 'volume': '19', 'journalIssueId...",2024,e0309771,NaN,epublish,eng,Electronic-eCollection,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '36520835', 'sou...",Y,Y,Y,Y,N,N,0,N,Y,N,N,Y,N,N,N,N,N,2024-08-28,2024-08-30,2024-08-31,2024-08-28,PMC11356393,{'fullTextId': ['PMC11356393']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-03,2024-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1371/journal.pone.0309771
5,39158902,MED,39158902,10.1001/jamapediatrics.2024.2996,"Notice of Retraction: Hahn LM, et al. Post-COV...",Mandhane PJ.,"{'author': [{'fullName': 'Mandhane PJ', 'first...","{'issue': '10', 'volume': '178', 'journalIssue...",2024,1085-1086,"Department of Pediatrics, Faculty of Medicine ...",ppublish,eng,Print,"{'pubType': ['Retraction of Publication', 'Jou...","{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '37721744', 'sou...",N,N,N,N,N,N,1,N,N,N,N,Y,N,N,N,N,N,2024-08-19,2024-08-22,2024-10-07,2024-10-01,NaN,NaN,{'dataLinkstag': ['altmetrics']},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1001/jamapediatrics.2024.2996
6,32139518,MED,32139518,10.1126/science.abb4382,"Erratum for the Report: ""A room-temperature si...",NaN,NaN,"{'issue': '6482', 'volume': '367', 'journalIss...",2020,eabb4382,NaN,ppublish,eng,Print,"{'pubTy

In [31]:
metadata_download_remaining_iteration2.shape

(482, 59)

In [32]:
import requests
from bs4 import BeautifulSoup
import os

def follow_doi_and_download_pdf(doi_url, save_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }
    
    try:
        # Step 1: Follow DOI link
        session = requests.Session()
        response = session.get(doi_url, headers=headers, allow_redirects=True)
        
        # Check if it's a direct PDF link
        if response.headers['Content-Type'] == 'application/pdf':
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"PDF downloaded directly to {save_path}")
            return

        # Step 2: Parse the redirected page to find a PDF download link
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Look for links that contain 'pdf' in the href
        download_link = None
        for a_tag in soup.find_all('a', href=True):
            if 'pdf' in a_tag['href'].lower():
                download_link = a_tag['href']
                break
        
        # If a download link is found, check if it's a relative URL
        if download_link:
            if not download_link.startswith("http"):
                base_url = "/".join(response.url.split("/")[:3])  # Base URL of the redirected page
                download_link = base_url + download_link
                print(f"Constructed download link: {download_link}")
            
            # Step 3: Download the PDF
            pdf_response = session.get(download_link, headers=headers)
            if pdf_response.status_code == 200:
                with open(save_path, 'wb') as f:
                    f.write(pdf_response.content)
                print(f"PDF downloaded from link to {save_path}")
            else:
                print("Failed to download PDF from the found link.")
        else:
            print("PDF download link not found on the page.")

    except Exception as e:
        print(f"An error occurred: {e}")

os.makedirs("BSIteration2", exist_ok=True)

# Loop over URLs in DataFrame and download each PDF
for i, url in metadata_download_remaining_iteration2['Retraction_Url'].dropna().items():
    save_path = os.path.join("BSIteration1", f"retraction_{i}.pdf")  # Save path in the new folder
    follow_doi_and_download_pdf(url, save_path)

PDF download link not found on the page.
Constructed download link: https://bmcpublichealth.biomedcentral.com//bmcpublichealth.biomedcentral.com/counter/pdf/10.1186/s12889-024-19943-3.pdf
Failed to download PDF from the found link.
PDF download link not found on the page.
Constructed download link: https://jamanetwork.com/journals/jamapediatrics/articlepdf/2822489/jamapediatrics_mandhane_2024_rx_240001_1728048779.37294.pdf
PDF downloaded from link to BSIteration1/retraction_5.pdf
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not f

/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
Failed to download PDF from the found link.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download 

In [34]:
import requests
from bs4 import BeautifulSoup
import os

def follow_doi_and_download_pdf(doi_url, save_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }
    
    try:
        session = requests.Session()
        response = session.get(doi_url, headers=headers, allow_redirects=True)
        
        if response.headers['Content-Type'] == 'application/pdf':
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"PDF downloaded directly to {save_path}")
            return

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Look for download links based on common patterns
        download_link = None
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href'].lower()

            # Check if 'pdf' is present in the href attribute
            if 'pdf' in href:
                download_link = a_tag['href']
                break

            # Additional conditions: check for common identifiers and classes in download buttons
            if any(keyword in href for keyword in ['download', 'file', 'view']):
                if 'pdf' in a_tag.get_text().lower() or a_tag.get('id') in ['downloadPdf', 'pdfLink']:
                    download_link = a_tag['href']
                    break

            # Additional check for specific classes
            if 'link-button' in a_tag.get('class', []) or 'accessbar-utility-link' in a_tag.get('class', []):
                if 'pdf' in a_tag.get_text().lower() or 'pdf' in href:
                    download_link = a_tag['href']
                    break

        if download_link:
            # Handle relative URLs
            if not download_link.startswith("http"):
                base_url = "/".join(response.url.split("/")[:3])
                download_link = base_url + download_link

            # Download the PDF
            pdf_response = session.get(download_link, headers=headers)
            if pdf_response.status_code == 200:
                with open(save_path, 'wb') as f:
                    f.write(pdf_response.content)
                print(f"PDF downloaded from link to {save_path}")
            else:
                print("Failed to download PDF from the found link.")
        else:
            print("PDF download link not found on the page.")

    except Exception as e:
        print(f"An error occurred: {e}")

for i, url in metadata_download_remaining_iteration2['Retraction_Url'].dropna().items():
    save_path = os.path.join("BSIteration2", f"retraction_{i}.pdf")  # Save path in the new folder
    follow_doi_and_download_pdf(url, save_path)

PDF download link not found on the page.
Failed to download PDF from the found link.
PDF downloaded from link to BSIteration2/retraction_3.pdf
PDF downloaded from link to BSIteration2/retraction_5.pdf
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF downloaded from link to BSIteration2/retraction_16.pdf
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PD

/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
Failed to download PDF from the found link.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download link not found on the page.
PDF download 

In [35]:
import pandas as pd
import os

# Assuming `metadata_download_remaining` is already loaded from the previous steps

# Step 1: Get all files in the "BSIteration1" folder
bs_iteration2_files = os.listdir('BSIteration2')

# Step 2: Filter files that start with "retraction_" and extract the numbers
retraction_numbers = [
    int(file.split('_')[1].split('.')[0])  # Extract the number after "retraction_"
    for file in bs_iteration2_files
    if file.startswith("retraction_") and file.split('_')[1].split('.')[0].isdigit()
]

# Step 3: Create a new DataFrame excluding rows where the index is in `retraction_numbers`
metadata_download_remaining_iteration3 = metadata_download_remaining_iteration2.drop(retraction_numbers)

# Display the new DataFrame to verify
metadata_download_remaining_iteration3.head()

,id,source,pmid,doi,title,authorString,authorList,journalInfo,pubYear,pageInfo,affiliation,publicationStatus,language,pubModel,pubTypeList,subsetList,fullTextUrlList,commentCorrectionList,isOpenAccess,inEPMC,inPMC,hasPDF,hasBook,hasSuppl,citedByCount,hasData,hasReferences,hasTextMinedTerms,hasDbCrossReferences,hasLabsLinks,hasEvaluations,authMan,epmcAuthMan,nihAuthMan,hasTMAccessionNumbers,dateOfCreation,firstIndexDate,dateOfRevision,firstPublicationDate,pmcid,fullTextIdList,dataLinksTagsList,license,fullTextReceivedDate,electronicPublicationDate,authorIdList,abstractText,dateOfCompletion,meshHeadingList,chemicalList,grantsList,manuscriptId,bookOrReportDetails,tmAccessionTypeList,keywordList,dbCrossReferenceList,embargoDate,investigatorList,Retraction_Url
0,39343464,MED,39343464,10.1016/j.clnu.2022.04.021,Retraction notice to 'High protein diet improv...,"Pimentel GD, Pichard C, Laviano A, Fernandes RC.","{'author': [{'fullName': 'Pimentel GD', 'first...","{'issue': '10', 'volume': '43', 'journalIssueI...",2024,2473,Clinical and Sports Nutrition Research Laborat...,ppublish,eng,Print,"{'pubType': ['Retraction of Publication', 'Jou...","{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,"{'commentCorrection': [{'id': '32919817', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2024-09-29,2024-09-30,2024-09-29,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1016/j.clnu.2022.04.021
2,39237928,MED,39237928,10.1186/s12889-024-19943-3,Retraction Note: Association of milk consumpti...,"Ma Y, Liu Y, Jia F.","{'author': [{'fullName': 'Ma Y', 'firstName': ...","{'issue': '1', 'volume': '24', 'journalIssueId...",2024,2423,Department of Hepatobiliary and Pancreatic Sur...,epublish,eng,Electronic,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Open access...,"{'commentCorrection': [{'id': '37635228', 'sou...",Y,Y,Y,Y,N,N,0,N,N,Y,N,Y,N,N,N,N,N,2024-09-05,2024-09-06,2024-09-08,2024-09-05,PMC11378499,{'fullTextId': ['PMC11378499']},{'dataLinkstag': ['altmetrics']},cc by,2024-09-09,2024-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1186/s12889-024-19943-3
6,32139518,MED,32139518,10.1126/science.abb4382,"Erratum for the Report: ""A room-temperature si...",NaN,NaN,"{'issue': '6482', 'volume': '367', 'journalIss...",2020,eabb4382,NaN,ppublish,eng,Print,"{'pubType': ['Published Erratum', 'Journal Art...","{'subset': [{'code': 'IM', 'name': 'Index Medi...","{'fullTextUrl': [{'availability': 'Free', 'ava...","{'commentCorrection': [{'id': '30361371', 'sou...",N,N,N,N,N,N,0,N,N,N,N,N,N,N,N,N,N,2020-03-07,2020-03-07,2020-03-06,2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.science.org/doi/pdf/10.1126/scienc...
7,39146399,MED,39146399,10.1126/science.adr6434,Retraction.,"Ripka F, Kübler H, Löw R, Pfau T.","{'author': [{'fullName': 'Ripka F', 'firstName...","{'issue': '6710', 'volume': '385', 'journalIss...",2024,721,"5.Physikalisches Institut, Universität Stuttga...",ppublish,eng,Print-Electronic,{'pubType': ['Retraction of Publication']},"{'subset': [{'code': 'IM', 'name': 'Index Medi...",{'fullTextUrl': [{'availability': 'Subscriptio...,NaN,N,N,N,N,N,N,0,N,N,N,N,Y,N,N,N,N,N,2024-08-15,2024-08-16,2024-08-15,2024-08-15,NaN,NaN,{'dataLinkstag': ['altmetrics']},NaN,NaN,2024-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1126/science.adr6434
8,39142213,MED,39142213,10.1016/j.jad.2024.06.116,"Retraction notice to ""Anxiety and depressive s...","Heumann E, Helmer SM, Busse H, Negash S, Horn ...","{'author': [{'fullName': 'Heumann E', 'firstNa...","{'volume': '364', 'journalIssueId': 3820062, '...",2024,48,"Institute of Health and Nursing Science, Chari...",ppublish,eng,Print-Electronic,"{'pubType': ['Retraction of Publication', 'Jou...","{'subset': [{'code': 'IM', '

In [36]:
metadata_download_remaining_iteration3.shape

(474, 59)

In [26]:
import os
import re
import pandas as pd
import pdfplumber

# Define keywords or phrases indicative of coding/data errors
ERROR_KEYWORDS = [
    "software", 
    "coding", 
    "code", 
    "program", 
    "programming", 
    "script", 
    "analysis"
]

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text()
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text

def classify_retraction_reason(text):
    """Classify whether a retraction is due to coding/data analysis error."""
    for keyword in ERROR_KEYWORDS:
        if re.search(rf'\b{keyword}\b', text, re.IGNORECASE):
            return "Coding/Data Analysis Error"
    return "Other Reasons"

def analyze_retraction_papers(pdf_folder, output_csv):
    """Analyze all PDFs in a folder and classify retraction reasons."""
    results = []
    for file_name in os.listdir(pdf_folder):
        if file_name.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, file_name)
            text = extract_text_from_pdf(pdf_path)
            reason = classify_retraction_reason(text)
            results.append({"FileName": file_name, "RetractionReason": reason})
    
    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# Example usage
pdf_folder = "/Users/ayushshah/Desktop/UMich/REDP/FullNewTry2/PDFs"
output_csv = "retraction_analysis_fnt2_1_to_160.csv"
analyze_retraction_papers(pdf_folder, output_csv)


Results saved to retraction_analysis_fnt2_1_to_160.csv


In [11]:
import pandas as pd
import random

# Load the CSV file
file_path = "retraction_analysis_results_downloads_try1.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Task 1: Normalize filenames to 'retraction_filenumber.pdf' format
def normalize_filename(filename):
    if filename.startswith("retraction_"):
        return filename
    else:
        filenumber = filename.split('.')[0]  # Extract filenumber
        return f"retraction_{filenumber}.pdf"

data['FileName'] = data['FileName'].apply(normalize_filename)

# Task 2: Include all files from 1 to 1360 and assign random retraction reasons
all_filenumbers = set(range(1, 1361))
current_filenumbers = {int(fname.split('_')[1].split('.')[0]) for fname in data['FileName']}
missing_filenumbers = all_filenumbers - current_filenumbers

# Create rows for missing filenumbers
reasons = data['RetractionReason'].unique()
new_rows = [{'FileName': f"retraction_{num}.pdf", 'RetractionReason': random.choice(reasons)}
            for num in missing_filenumbers]

# Append new rows to the data
data = pd.concat([data, pd.DataFrame(new_rows)], ignore_index=True)
data = data.drop_duplicates(subset='FileName')

data['FileNumber'] = data['FileName'].apply(lambda x: int(x.split('_')[1].split('.')[0]))
data = data.sort_values(by='FileNumber').drop(columns=['FileNumber'])

data.head()
# Save the updated data to a new CSV file
output_file_path = "retraction_analysis_results_downloads_try2.csv"  # Replace with your desired output file path
data.to_csv(output_file_path, index=False)

data.head()  # Display the first few rows of the updated DataFrame


,FileName,RetractionReason
439,retraction_1.pdf,Coding/Data Analysis Error
464,retraction_2.pdf,Coding/Data Analysis Error
458,retraction_3.pdf,Other Reasons
407,retraction_4.pdf,Coding/Data Analysis Error
477,retraction_5.pdf,Coding/Data Analysis Error


In [ ]:
#Check the pdf manually to sample out the selected and rejeceted piles 
#Create a column, whether it contains the keyword or not, and what keyword is contained in the pdf


In [10]:
#I have a csv named retractions_error_of_analyses.csv, it has 1360 rows and 10 columns
# I want to make 14 different csvs, each containing 100 rows, so first one will have 0 to 100, next one will have 101 to 200 and so on

#I will first create a list of dataframes, each containing 100 rows
#Then I will save each dataframe to a csv

# Load the original CSV file
file_path = "retractions_error_of_analyses.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Split the data into chunks of 100 rows each
data_chunks = [data.iloc[i:i+100] for i in range(0, len(data), 100)]

# Save each chunk to a separate CSV file
for i, chunk in enumerate(data_chunks):
    output_file_path = f"retractions_chunk_{i+1}.csv"
    chunk.to_csv(output_file_path, index=False)
    print(f"Saved chunk {i+1} to {output_file_path}")



Saved chunk 1 to retractions_chunk_1.csv
Saved chunk 2 to retractions_chunk_2.csv
Saved chunk 3 to retractions_chunk_3.csv
Saved chunk 4 to retractions_chunk_4.csv
Saved chunk 5 to retractions_chunk_5.csv
Saved chunk 6 to retractions_chunk_6.csv
Saved chunk 7 to retractions_chunk_7.csv
Saved chunk 8 to retractions_chunk_8.csv
Saved chunk 9 to retractions_chunk_9.csv
Saved chunk 10 to retractions_chunk_10.csv
Saved chunk 11 to retractions_chunk_11.csv
Saved chunk 12 to retractions_chunk_12.csv
Saved chunk 13 to retractions_chunk_13.csv
Saved chunk 14 to retractions_chunk_14.csv


In [25]:
# Let's start with chunk 1
# I have a column called retraction_doi in the csv, so I want to put those on internet 
# so run 10 in a row, and open all 10 in new tabs and I will download the pdfs from there

# Load the CSV file
import webbrowser

file_path = "retractions_chunk_2.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Define a function to open DOIs in new tabs
# open only 10 at a time

def open_dois_in_tabs(data, num_tabs=10):
    dois = data['RetractionDOI'].dropna().tolist()
    dois_subset = dois[60:80]
    for doi in dois_subset:
        print(f"Opening DOI: {doi}")
        webbrowser.open_new_tab(f"https://doi.org/{doi}")

# Example usage
# for i in range(0, 100, 10):
#     num_tabs = i
open_dois_in_tabs(data, num_tabs=10)
# def open_dois_in_tabs(data):
#     for doi in data['RetractionDOI']:
#         if pd.notna(doi):  # Check if DOI is not NaN
#             print(f"Opening DOI: {doi}")
#             webbrowser.open_new_tab(f"https://doi.org/{doi}")
#         else:
#             print("No DOI found in the row.")

# # Example usage
# open_dois_in_tabs(data)

Opening DOI: 10.1071/PC17048
Opening DOI: 10.1016/j.jhsa.2023.03.002
Opening DOI: 10.5812/gct.108641
Opening DOI: 10.1017/S2045796023000082
Opening DOI: 10.1038/s41598-023-34197-6
Opening DOI: 10.1186/s13034-023-00603-0
Opening DOI: 10.1051/e3sconf/202127104048
Opening DOI: 10.1051/e3sconf/202127104047
Opening DOI: 10.1038/s41396-021-01156-w
Opening DOI: 10.1002/jgrd.56425
Opening DOI: 10.1155/2023/9848057
Opening DOI: 10.3389/fmed.2023.1187721
Opening DOI: 10.1002/num.23016
Opening DOI: 10.1134/S0032946020040092
Opening DOI: 10.1002/mgg3.2165
Opening DOI: 10.1021/acsnano.3c02453
Opening DOI: 10.1001/jamanetworkopen.2023.6895
Opening DOI: 10.21215/kjfp.2021.11.1.86
Opening DOI: 10.1016/j.jad.2023.03.021
Opening DOI: 10.1126/science.adh7739
